In [1]:
import numpy as np
import pandas as pd
import os 
import time
import deepdish.io as dd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler

from utils.inference import SVI
import utils.models as m
import utils.datasets as d
from utils.generative_models import SemiSupervisedGenerativeModel, Generative_Model_Trainer
from utils.layers import *
from utils.inception import *

In [2]:
X_s = np.vstack((np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt1.npy"),
               np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt2.npy")))
y_s = np.load("data/sequenced_data_for_VAE_length-160_stride-10_targets.npy")

In [3]:
X_u = np.vstack((np.load("data/unlabeled_sequences_part_1.npy"), np.load("data/unlabeled_sequences_part_2.npy")))

In [4]:
print(X_s.shape)
print(X_u.shape)

(83680, 160)
(97400, 160)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X_s), np.array(y_s), test_size=0.2, random_state=666)

In [6]:
scaler = RobustScaler()
scaler.fit(np.vstack((np.array(X_s),np.array(X_u))))

X_sup= scaler.transform(X_train)
X_unsup = scaler.transform(np.array(X_u))
X_test = scaler.transform(X_test)

In [7]:
sup = d.H_alphaSequences(X_sup, y_s)
test = d.H_alphaSequences(X_test, y_test)

# Encoder & Decoder

In [8]:
ORIGINAL_DIM = 160
LATENT_DIM =16
Y_DIM = 4

In [9]:
class Encoder_with_y(nn.Module):
    def __init__(self, ORIGINAL_DIM, LATENT_DIM, Y_DIM):
        super(Encoder_with_y,self).__init__()
        self.conv_enc = nn.Sequential(
            Reshape(out_shape=(1, ORIGINAL_DIM)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ReLU(),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False),#n*16*78->n*32*38
            nn.BatchNorm1d(num_features=32),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False),#n*32*38->n*64*18
            nn.BatchNorm1d(num_features=64),
            nn.ReLU(),
            Flatten(out_features=64*18))
        self.fc = nn.Sequential(
            nn.Linear(in_features=64*18 +Y_DIM, out_features=256),
            nn.ReLU(),
            VariationalLayer(in_features=256, out_features=LATENT_DIM, return_KL=False))
        
    def forward(self, X,y):
        cn = self.conv_enc(X)
        X = torch.cat([cn,y], axis=1)
        #print(X.shape)
        return self.fc(X)

In [10]:
encoder = nn.Sequential(
    Reshape(out_shape=(1, ORIGINAL_DIM)),
    nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
    nn.ReLU(),
    nn.Conv1d(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False),#n*16*78->n*32*38
    nn.BatchNorm1d(num_features=32),
    nn.ReLU(),
    nn.Conv1d(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False),#n*32*38->n*64*18
    nn.BatchNorm1d(num_features=64),
    nn.ReLU(),
    Flatten(out_features=64*18),
    nn.Linear(in_features=64*18, out_features=256),
    nn.ReLU(),
    VariationalLayer(in_features=256, out_features=LATENT_DIM, return_KL=False)
    )

In [11]:
#encoder = Encoder_with_y(ORIGINAL_DIM, LATENT_DIM, Y_DIM)
#encoder

In [12]:
decoder = nn.Sequential(
    nn.Linear(in_features=LATENT_DIM+Y_DIM, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=64*18),
    Reshape(out_shape=(64,18)),
    nn.BatchNorm1d(num_features=64),
    nn.ReLU(),
    nn.ConvTranspose1d(in_channels=64, out_channels=32, kernel_size=4, stride=2, padding=0),
    nn.BatchNorm1d(num_features=32),
    nn.ReLU(),
    nn.ConvTranspose1d(in_channels=32, out_channels=16, kernel_size=4, stride=2, padding=0),
    #nn.BatchNorm1d(num_features=16),
    nn.ReLU(),
    ConvTransposeDecoderOutput(
        in_channels=16, 
        in_features=16*78, 
        out_features=ORIGINAL_DIM, 
        kernel_size=6, 
        stride=2
        )
    )

In [13]:
decoder2 = nn.Sequential(
    nn.Linear(in_features=LATENT_DIM+Y_DIM, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=64*18),
    Reshape(out_shape=(64,18)),
    nn.BatchNorm1d(num_features=64),
    nn.ReLU(),
    nn.ConvTranspose1d(in_channels=64, out_channels=32, kernel_size=4, stride=2, padding=0),
    nn.BatchNorm1d(num_features=32),
    nn.ReLU(),
    nn.ConvTranspose1d(in_channels=32, out_channels=16, kernel_size=4, stride=2, padding=0),
    #nn.BatchNorm1d(num_features=16),
    nn.ReLU(),
    nn.ConvTranspose1d(in_channels=16, out_channels=1, kernel_size=6, stride=2, padding=0),
    Flatten(out_features=160)
    )

# Classifier

In [14]:
clf = nn.Sequential(
            Reshape(out_shape=(1,160)),
            InceptionBlock(
                in_channels=1, 
                n_filters=32, 
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                use_residual=True,
                activation=nn.ReLU()
            ),
            InceptionBlock(
                in_channels=32*4, 
                n_filters=32, 
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                use_residual=True,
                activation=nn.ReLU()
            ),
            nn.AdaptiveAvgPool1d(output_size=1),
            Flatten(out_features=32*4*1),
            nn.Linear(in_features=4*32*1, out_features=4)
)

In [15]:
clf2 = nn.Sequential(
            Reshape(out_shape=(1,160)),
            InceptionBlock(
                in_channels=1, 
                n_filters=32, 
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                use_residual=True,
                activation=nn.ReLU()
            ),
            Flatten(out_features=32*4*160),
            nn.Linear(in_features=4*32*160, out_features=4)
)

In [24]:
clf3 = nn.Sequential(
    Reshape(out_shape=(1, ORIGINAL_DIM)),
    nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
    nn.ReLU(),
    nn.Conv1d(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False),#n*16*78->n*32*38
    nn.BatchNorm1d(num_features=32),
    nn.ReLU(),
    nn.Conv1d(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False),#n*32*38->n*64*18
    nn.BatchNorm1d(num_features=64),
    nn.ReLU(),
    Flatten(out_features=64*18),
    nn.Linear(in_features=64*18, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=Y_DIM)
    )

In [25]:
class MSE(nn.Module):
    def __init__(self, reduction="mean"):
        super(MSE, self).__init__()
        self.reduction = reduction

    def forward(self, y_pred, y_true):
        if self.reduction=="mean":
            return - (torch.mean(torch.sum((y_true-y_pred).pow(2), axis=1)))
        if self.reduction=="none":
            return - (torch.sum((y_true-y_pred).pow(2), axis=1))


# Generative Model

In [68]:
model = SemiSupervisedGenerativeModel(
                            encoder=nn.Sequential(
                                nn.Linear(in_features=160, out_features=400),
                                nn.ELU(),
                                nn.Linear(in_features=400, out_features=200),
                                nn.ELU(),
                                VariationalLayer(in_features=200, out_features=30, return_KL=False)), 
                            decoder=nn.Sequential(
                                nn.Linear(in_features=30+4, out_features=200),
                                nn.ELU(),
                                nn.Linear(in_features=200, out_features=400),
                                nn.ELU(),
                                VariationalDecoderOutput(in_features=400, out_features=160)
                                ),
                            classifier=nn.Sequential(
                                nn.Linear(in_features=160, out_features=400),
                                nn.ELU(),
                                nn.Linear(in_features=400, out_features=200),
                                nn.ELU(),
                                nn.Linear(in_features=200, out_features=4)
                                ),
                            y_dim=Y_DIM,
                            include_y=False
                            )

# Define Model Trainer

In [69]:
gmt = Generative_Model_Trainer(
    model=model, 
    optimizer=torch.optim.Adam, 
    scheduler=None, 
    lr=1e-3, 
    tensorboard=True, 
    model_name="InceptionTime_mod+VAE_lr-1e-3_bs-512", 
    verbose=True)

Using device cuda:0
Generative_Model_Trainer(
  (model): SemiSupervisedGenerativeModel(
    (encoder): Sequential(
      (0): Linear(in_features=160, out_features=400, bias=True)
      (1): ELU(alpha=1.0)
      (2): Linear(in_features=400, out_features=200, bias=True)
      (3): ELU(alpha=1.0)
      (4): VariationalLayer(
        (mu): Linear(in_features=200, out_features=30, bias=True)
        (rho): Linear(in_features=200, out_features=30, bias=True)
        (softplus): Softplus(beta=1, threshold=20)
      )
    )
    (decoder): Sequential(
      (0): Linear(in_features=34, out_features=200, bias=True)
      (1): ELU(alpha=1.0)
      (2): Linear(in_features=200, out_features=400, bias=True)
      (3): ELU(alpha=1.0)
      (4): VariationalDecoderOutput(
        (mu): Linear(in_features=400, out_features=160, bias=True)
        (rho): Linear(in_features=400, out_features=1, bias=True)
        (softplus): Softplus(beta=1, threshold=20)
      )
    )
    (classifier): Sequential(
      (

In [70]:
gmt(epochs=range(20), supervised_dataset=sup, unsupervised_dataset = X_unsup, validation_dataset=test, batch_size=512)

Epoch [1/20], average_loss:-131.2817, validation_loss:-230.6921, val_accuracy:0.6245
Epoch [2/20], average_loss:-275.2300, validation_loss:-285.7618, val_accuracy:0.6243
Epoch [3/20], average_loss:-326.9514, validation_loss:-305.2858, val_accuracy:0.6244
Epoch [4/20], average_loss:-337.1337, validation_loss:-316.5893, val_accuracy:0.6244
Epoch [5/20], average_loss:-350.3854, validation_loss:-334.0041, val_accuracy:0.6244
Epoch [6/20], average_loss:-356.4182, validation_loss:-340.2420, val_accuracy:0.6244
Epoch [7/20], average_loss:-345.3538, validation_loss:-353.6735, val_accuracy:0.6244
Epoch [8/20], average_loss:-378.8522, validation_loss:-360.2030, val_accuracy:0.6244
Epoch [9/20], average_loss:nan, validation_loss:nan, val_accuracy:0.2530
Epoch [10/20], average_loss:nan, validation_loss:nan, val_accuracy:0.2530
Epoch [11/20], average_loss:nan, validation_loss:nan, val_accuracy:0.2530


KeyboardInterrupt: 

tensor([1.3863, 1.3863, 1.3863])

In [74]:
sup.X.shape

torch.Size([66944, 160])